# **Pydantic**

Pydantic ei muuda Pythonit staatiliselt tüübikontrollitud keelest, nagu Java, kuid pakub tugevat tüübiannotatsioonide süsteemi ja andmete valideerimist. See aitab parandada koodi täpsust ja vältida vigade tekkimist, rakendades mõningaid staatilise tüübikontrolli omadusi dünaamilise keele kontekstis.

# 1. Tüübikontroll ja Andmete Valideerimine
Pydantic on andmemudelite valideerimise ja serialiseerimise tööriist, mis kasutab Python 3.6+ tüübiannotatsioone andmete valideerimiseks ja automaatseks konverteerimiseks. See aitab kindlaks teha, et andmed järgivad määratud tüüpe ja reegleid.

**Näide: Andmete Valideerimine**

In [1]:
from pydantic import BaseModel, conint

class User(BaseModel):
    name: str
    age: conint(ge=0)  # Age must be a non-negative integer

# Valid data
user = User(name="Alice", age=30)

# Invalid data (raises a validation error)
try:
    invalid_user = User(name="Bob", age=-5)
except ValueError as e:
    print(e)


1 validation error for User
age
  Input should be greater than or equal to 0 [type=greater_than_equal, input_value=-5, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/greater_than_equal


# 2. Tüübihinnangud ja Automaatsed Konversioonid
Pydantic võib automaatselt konverteerida andmeid õigesse vormingusse vastavalt tüübiannotatsioonidele. Näiteks, kui sisendiks on tekstiline kuupäev, saab Pydantic selle automaatselt datetime objektiks konverteerida.

**Näide: Automaatne Konversioon**

In [2]:
from pydantic import BaseModel
from datetime import datetime

class Event(BaseModel):
    name: str
    date: datetime  # Expect a datetime object

event = Event(name="Conference", date="2024-09-01T10:00:00")
print(event.date)  # Output will be a datetime object


2024-09-01 10:00:00


# 3. Tüübihinnangud ja Kasutaja Defineeritud Väärtused
Pydantic võimaldab defineerida kompleksseid andmemudeleid, kus saab määratleda keerukamaid struktuure ja reegleid. Näiteks, saab luua andmemudeleid, kus alammudelite tüübid on täpselt määratletud.

**Näide: Kompleksne Andmemudel**

In [3]:
from pydantic import BaseModel
from typing import List

class Address(BaseModel):
    street: str
    city: str

class Person(BaseModel):
    name: str
    addresses: List[Address]  # List of Address objects

# Valid data
person = Person(name="John", addresses=[{"street": "123 Elm St", "city": "Somewhere"}])

# Invalid data (raises a validation error)
try:
    invalid_person = Person(name="Jane", addresses=[{"street": "456 Oak St", "city": 123}])
except ValueError as e:
    print(e)


1 validation error for Person
addresses.0.city
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/string_type


# 4. Tüübi Kaitsmine ja Error Handling
Pydantic võib aidata vältida tavalisi tüübi- ja andmete valideerimisvigu, mida võidakse täiendavalt kontrollida ja käsitleda, enne kui andmed jõuavad rakenduse äriloogikasse.

**Näide: Error Handling**

In [4]:
from pydantic import BaseModel, ValidationError

class Product(BaseModel):
    name: str
    price: float

try:
    product = Product(name="Gadget", price="free")  # Invalid price type
except ValidationError as e:
    print(e.json())


[{"type":"float_parsing","loc":["price"],"msg":"Input should be a valid number, unable to parse string as a number","input":"free","url":"https://errors.pydantic.dev/2.8/v/float_parsing"}]


 Luues klassis @validator dekoraatoreid, mis võimaldavad valideerida ja muuta väärtusi enne, kui need mudelisse sisestatakse.

In [5]:
from pydantic import BaseModel, validator, ValidationError

class Product(BaseModel):
    name: str
    price: float

    @validator('price', pre=True)
    def parse_price(cls, value):
        # Kui väärtus on string "free", muuda see 0-ks
        if value == 'free':
            return 0.0
        # Proovi muundada väärtus arvuks, kui see ei ole juba float
        try:
            return float(value)
        except ValueError:
            raise ValueError(f"Invalid price value: {value}")

# Näide kehtivatest andmetest
try:
    product1 = Product(name="Gadget", price="free")  # "free" muudetakse 0-ks
    print(product1)
except ValidationError as e:
    print(e)

# Näide vigaste andmetega
try:
    product2 = Product(name="Gadget", price="not_a_number")  # Tõstatab ValueError
except ValidationError as e:
    print(e)


name='Gadget' price=0.0
1 validation error for Product
price
  Value error, Invalid price value: not_a_number [type=value_error, input_value='not_a_number', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/value_error


<ipython-input-5-334f3f147b1f>:7: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  @validator('price', pre=True)


# Näide Pydantic mudelist, mis kontrollib SQL-päringu olemasolu
Siin on lihtne näide, kuidas luua Pydantic mudel, mis kasutab kohandatud validatsioonimeetodit, et kontrollida, kas sisend on SQL-päring või sisaldab SQL-päringu elemente:

In [8]:
from pydantic import BaseModel, validator, ValidationError

class SQLQuery(BaseModel):
    query: str

    @validator('query')
    def validate_sql_query(cls, value):
        # Näide SQL-päringu mustrist, mis kontrollib tavalisi SQL-võtmesõnu
        sql_keywords = ['SELECT', 'INSERT', 'UPDATE', 'DELETE', 'FROM', 'WHERE', 'JOIN']
        if any(keyword in value.upper() for keyword in sql_keywords):
            raise ValueError("The input appears to be an SQL query.")
        return value

# Näide kehtiva sisendiga
try:
    query = SQLQuery(query="SELECT * FROM users WHERE id = 1")
except ValidationError as e:
    print(e)

# Näide mittesobiva sisendiga
try:
    query = SQLQuery(query="This is just a regular string.")
    print(query)
except ValidationError as e:
    print(e)


1 validation error for SQLQuery
query
  Value error, The input appears to be an SQL query. [type=value_error, input_value='SELECT * FROM users WHERE id = 1', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/value_error
query='This is just a regular string.'


<ipython-input-8-cfc7c29c99ac>:6: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  @validator('query')


Kujuta ette, et soovid kontrollida, kas päring sisaldab SQL-klausleid nagu SELECT ja FROM, kasutades regulaaravaldisi.

In [7]:
from pydantic import BaseModel, validator, ValidationError
import re

class SQLQuery(BaseModel):
    query: str

    @validator('query')
    def validate_sql_query(cls, value):
        # Define regular expression patterns for SQL keywords
        sql_pattern = re.compile(r'\b(SELECT|INSERT|UPDATE|DELETE|FROM|WHERE|JOIN)\b', re.IGNORECASE)
        if sql_pattern.search(value):
            raise ValueError("The input appears to be an SQL query.")
        return value

# Näide kehtiva sisendiga
try:
    query = SQLQuery(query="SELECT * FROM users WHERE id = 1")
except ValidationError as e:
    print(e)

# Näide mittesobiva sisendiga
try:
    query = SQLQuery(query="This is just a regular string.")
    print(query)
except ValidationError as e:
    print(e)


1 validation error for SQLQuery
query
  Value error, The input appears to be an SQL query. [type=value_error, input_value='SELECT * FROM users WHERE id = 1', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/value_error
query='This is just a regular string.'


<ipython-input-7-8a7692f7ee3e>:7: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  @validator('query')


# Näide: Algandmete Kontrollimine
1. Pydantic Mudeli Loomine: kuidas kasutada Pydanticut algarvude kontrollimiseks:

Seda lähenemist saab kasutada igasuguste keerukate valideerimisnõuete täitmiseks Pydanticus.

In [12]:
from pydantic import BaseModel, validator
from typing import List

class PrimeNumberValidator(BaseModel):
    number: int

    @validator('number')
    def check_prime(cls, value):
        if value <= 1:
            raise ValueError(f"{value} is not a prime number")
        for i in range(2, int(value**0.5) + 1):
            if value % i == 0:
                raise ValueError(f"{value} is not a prime number")
        return value

class NumbersModel(BaseModel):
    numbers: List[PrimeNumberValidator]

    @validator('numbers', each_item=True)
    def check_prime_in_list(cls, value):
        return value  # Each item is already validated by PrimeNumberValidator


<ipython-input-12-4c8db7bdffb1>:7: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  @validator('number')
<ipython-input-12-4c8db7bdffb1>:19: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  @validator('numbers', each_item=True)


In [13]:
try:
    # Proovime kehtiva andmestruktuuriga
    valid_data = NumbersModel(numbers=[PrimeNumberValidator(number=2),
                                       PrimeNumberValidator(number=3),
                                       PrimeNumberValidator(number=5)])
    print("Valid data:", valid_data)

    # Proovime kehtetu andmestruktuuriga
    invalid_data = NumbersModel(numbers=[PrimeNumberValidator(number=4),
                                         PrimeNumberValidator(number=7)])
except ValueError as e:
    print("Validation error:", e)


Valid data: numbers=[PrimeNumberValidator(number=2), PrimeNumberValidator(number=3), PrimeNumberValidator(number=5)]
Validation error: 1 validation error for PrimeNumberValidator
number
  Value error, 4 is not a prime number [type=value_error, input_value=4, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/value_error
